# Federated Protein Embeddings and Task Model Fitting with BioNeMo

<div class="alert alert-block alert-info"> <b>NOTE</b> This notebook was tested on a single A1000 GPU and is compatible with BioNeMo Framework v2.2. To leverage additional or higher-performance GPUs, you can modify the configuration files and simulation script to accommodate multiple devices and increase thread utilization respectively. </div>

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model in a federated learning (FL) setting. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This example is based on NVIDIA BioNeMo Service [example](https://github.com/NVIDIA/BioNeMo/blob/main/examples/service/notebooks/task-fitting-predictor.ipynb) 
but runs inference locally (on the FL clients) instead of using BioNeMo's cloud API.

This notebook will walk you through the task fitting workflow in the following sections:

* Dataset sourcing & Data splitting
* Federated embedding extraction
* Training an MLP to predict subcellular location

## Setup

Ensure that you have read through the Getting Started section, can run the BioNeMo Framework docker container, and have configured the NGC Command Line Interface (CLI) within the container. It is assumed that this notebook is being executed from within the container.

<div class="alert alert-block alert-info"> <b>NOTE</b> Some of the cells below generate long text output.  We're using <pre>%%capture --no-display --no-stderr cell_output</pre> to suppress this output.  Comment or delete this line in the cells below to restore full output.</div>

### Import and install all required packages

In [ ]:
%%capture --no-display --no-stderr cell_output
! pip install nvflare~=2.5
! pip install biopython
! pip install scikit-learn
! pip install matplotlib

import io
import numpy as np
import os
import pickle
import re
import requests
import split_data

from Bio import SeqIO
from importlib import reload
from nvflare import SimulatorRunner  
from split_data import split

### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, each FL client can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein my be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [ ]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [ ]:
# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

### Download Model Checkpoints

The following code will download the pre-trained model, `esm2n/650m:2.0`, from the NGC registry:

In [ ]:
from bionemo.core.data.load import load

checkpoint_path = load("esm2/650m:2.0", source="ngc")
print(checkpoint_path)

### Data splitting
Next, we prepare the data for simulating federated learning using `n_clients`.

In [20]:
n_clients = 3
# limiting to the proteins with sequence length<512 for embedding queries
MAX_SEQUENCE_LEN = 512
seed=0
out_dir = "/tmp/data/mixed_soft"
split_alpha = 1.0  # moderate label heterogeneity of alpha=1.0

reload(split_data)

np.random.seed(seed)

# Extract meta data and split
data = []
for i, x in enumerate(proteins):
        if len(str(x.seq)) > MAX_SEQUENCE_LEN:
            continue
            
        entry = {key: value for key, value in re.findall(r"([A-Z_]+)=(-?[A-z0-9]+[.0-9]*)", x.description)}
        entry["sequences"] = str(x.seq)
        entry["id"] = str(i)
        entry["labels"] = entry["TARGET"]
       
        data.append(entry)
print(f"Read {len(data)} valid sequences.")
               
# Split the data and save for each client
# Note, test_data is kept the same on each client and is not split
# `concat=False` is used for SCL experiments (see ../downstream/scl)
split(proteins=data, num_sites=n_clients, split_dir=out_dir, alpha=split_alpha, concat=False)  
# `concat=True` is used for separate inference + MLP classifier in this notebook
split(proteins=data, num_sites=n_clients, split_dir=out_dir, alpha=split_alpha, concat=True)  

Read 8619 valid sequences.
Partition protein dataset with 10 classes into 3 sites with Dirichlet sampling under alpha 1.0
{'site-1': {'Cell_membrane': 137,
            'Cytoplasm': 311,
            'Endoplasmic_reticulum': 123,
            'Extracellular': 356,
            'Golgi_apparatus': 43,
            'Lysosome': 39,
            'Mitochondrion': 250,
            'Nucleus': 453,
            'Peroxisome': 23,
            'Plastid': 110},
 'site-2': {'Cell_membrane': 217,
            'Cytoplasm': 491,
            'Endoplasmic_reticulum': 195,
            'Extracellular': 562,
            'Golgi_apparatus': 70,
            'Lysosome': 63,
            'Mitochondrion': 396,
            'Nucleus': 716,
            'Peroxisome': 38,
            'Plastid': 174},
 'site-3': {'Cell_membrane': 160,
            'Cytoplasm': 361,
            'Endoplasmic_reticulum': 144,
            'Extracellular': 414,
            'Golgi_apparatus': 51,
            'Lysosome': 47,
            'Mitochondrion'

### Federated embedding extraction
Running inference of the ESM-1nv model to extract embeddings requires a GPU with at least 12 GB memory. Here we run inference on each client sequentially using one thread to preserve GPU memory.

First, copy the model into the job folder

In [ ]:
!cp -r $checkpoint_path jobs/embeddings/app/models/.

In [23]:
simulator = SimulatorRunner(
    job_folder="jobs/embeddings",
    workspace="/tmp/nvflare/bionemo/embeddings",
    n_clients=n_clients,
    threads=1  # due to memory constraints, we run the client execution sequentially in one thread
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2024-12-04 03:08:37,653 - SimulatorRunner - INFO - Create the Simulator Server.
2024-12-04 03:08:37,659 - CoreCell - INFO - server: creating listener on tcp://0:35267
2024-12-04 03:08:37,673 - CoreCell - INFO - server: created backbone external listener for tcp://0:35267
2024-12-04 03:08:37,674 - ConnectorManager - INFO - 35605: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2024-12-04 03:08:37,676 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:2719] is starting
2024-12-04 03:08:38,179 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:2719
2024-12-04 03:08:38,180 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:35267] is starting
2024-12-04 03:08:38,306 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 41205
2024-12-04 03:08:38,308 - SimulatorRunner - INFO - Deploy the Apps.
2024-12-04 03:08:43,496 - SimulatorRunner - INFO - Create t

### Inspecting the embeddings and labels
Embeddings returned from the BioNeMo model are vectors of fixed size for each input sequence. In other words, if we input 10 sequences, we will obtain a matrix `10xD`, where `D` is the size of the embedding (in the case of ESM2-650m, `D=1280`). At a glance, these real-valued vector embeddings don't show any obvious features (see the printout in the next cell). But these vectors do contain information that can be used in downstream models to reveal properties of the protein, for example the subcellular location as we'll explore below.

In [36]:
# load embeddings from site-1
import torch
results = torch.load("/tmp/nvflare/bionemo/embeddings/site-1/simulate_job/app_site-1/inference_results.pt")

for key, val in results.items():
    if val is not None:
        print(f'{key}\t{val.shape}')

protein_embeddings = results['embeddings']
print(f"Loaded {len(protein_embeddings)} embeddings from site-1.")

for i in range(4):
    x = protein_embeddings[i]
    print(f"{i+1}. embedding: range {x.min():.2f}-{x.max():.2f}, mean={x.mean():.2f}, shape={x.shape}")

token_logits	torch.Size([1024, 3543, 128])
input_ids	torch.Size([3543, 1024])
embeddings	torch.Size([3543, 1280])
Loaded 3543 embeddings from site-1.
1. embedding: range -5.90-0.35, mean=-0.00, shape=torch.Size([1280])
2. embedding: range -5.75-0.30, mean=-0.00, shape=torch.Size([1280])
3. embedding: range -6.08-0.31, mean=-0.00, shape=torch.Size([1280])
4. embedding: range -7.75-0.65, mean=-0.00, shape=torch.Size([1280])


Let's enumerate the labels corresponding to potential subcellular locations.

In [38]:
# Let's also print all the labels

labels = set([entry['labels'] for entry in data])

for i, label in enumerate(labels):
    print(f"{i+1}. {label.replace('_', ' ')}")

1. Cytoplasm
2. Plastid
3. Cell membrane
4. Endoplasmic reticulum
5. Extracellular
6. Nucleus
7. Golgi apparatus
8. Lysosome
9. Peroxisome
10. Mitochondrion


### Training a MLP to predict subcellular location
To be able to classify proteins for their subcellular location, we train a simple scikit-learn Multi-layer Perceptron (MPL) classifier using Federated Averaging ([FedAvg](https://arxiv.org/abs/1602.05629)). The MLP model uses a network of hidden layers to fit the input embedding vectors to the model classes (the cellular locations above). In the simulation below, we define the MLP to use the Adam optimizer with a network of (512, 256, 128) hidden layers, defining a random state (or seed) for reproducibility, and trained for 30 rounds of FedAvg (see [config_fed_server.json](./jobs/fedavg/app/config/config_fed_server.json)). 

We can use the same configuration also to simulate local training where each client is only training with their own data by setting `os.environ["SIM_LOCAL"] = "True"`. Our [BioNeMoMLPLearner](./jobs/fedavg/app/custom/bionemo_mlp_learner.py) will then ignore the global weights coming from the server.

### Local training

In [57]:
os.environ["SIM_LOCAL"] = "True"

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace=f"/tmp/nvflare/bionemo/local_alpha{split_alpha}",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2024-12-04 19:48:21,311 - SimulatorRunner - INFO - Create the Simulator Server.
2024-12-04 19:48:21,317 - CoreCell - INFO - server: creating listener on tcp://0:51213
2024-12-04 19:48:21,334 - CoreCell - INFO - server: created backbone external listener for tcp://0:51213
2024-12-04 19:48:21,335 - ConnectorManager - INFO - 54084: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2024-12-04 19:48:21,337 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:63750] is starting
2024-12-04 19:48:21,839 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:63750
2024-12-04 19:48:21,841 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:51213] is starting
2024-12-04 19:48:21,967 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 36085
2024-12-04 19:48:21,968 - SimulatorRunner - INFO - Deploy the Apps.
2024-12-04 19:48:21,980 - SimulatorRunner - INFO - Create

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


2024-12-04 19:48:22,363 - BioNeMoMLPModelPersistor - INFO - [identity=simulator_server, run=simulate_job]: MLPClassifier coefficients [(1280, 512), (512, 256), (256, 128), (128, 10)], intercepts [(512,), (256,), (128,), (10,)]
2024-12-04 19:48:22,364 - AuxRunner - INFO - registered aux handler for topic fed.event
2024-12-04 19:48:22,369 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: starting workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) ...
2024-12-04 19:48:22,370 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Initializing ScatterAndGather workflow.
2024-12-04 19:48:22,371 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) started
2024-12-04 19:48:22,376 - ScatterAndGather - INFO - [identity=simulator_server,

### Federated learning

In [58]:
os.environ["SIM_LOCAL"] = "False"

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace=f"/tmp/nvflare/bionemo/fedavg_alpha{split_alpha}",
    n_clients=n_clients,
    threads=n_clients
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2024-12-04 19:56:47,498 - SimulatorRunner - INFO - Create the Simulator Server.
2024-12-04 19:56:47,503 - CoreCell - INFO - server: creating listener on tcp://0:42837
2024-12-04 19:56:47,518 - CoreCell - INFO - server: created backbone external listener for tcp://0:42837
2024-12-04 19:56:47,520 - ConnectorManager - INFO - 57645: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2024-12-04 19:56:47,522 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:3444] is starting
2024-12-04 19:56:48,024 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:3444
2024-12-04 19:56:48,026 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:42837] is starting
2024-12-04 19:56:48,151 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 60259
2024-12-04 19:56:48,153 - SimulatorRunner - INFO - Deploy the Apps.
2024-12-04 19:56:48,167 - SimulatorRunner - INFO - Create t

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


2024-12-04 19:56:48,543 - BioNeMoMLPModelPersistor - INFO - [identity=simulator_server, run=simulate_job]: MLPClassifier coefficients [(1280, 512), (512, 256), (256, 128), (128, 10)], intercepts [(512,), (256,), (128,), (10,)]
2024-12-04 19:56:48,547 - AuxRunner - INFO - registered aux handler for topic fed.event
2024-12-04 19:56:48,552 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: starting workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) ...
2024-12-04 19:56:48,554 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Initializing ScatterAndGather workflow.
2024-12-04 19:56:48,555 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) started
2024-12-04 19:56:48,560 - ScatterAndGather - INFO - [identity=simulator_server,

### TensorBoard Visualization
You can visualize the training progress using TensorBoard
```
tensorboard --logdir /tmp/nvflare/bionemo
```

An example of local (red) vs federated (blue) training is shown below.

![TensorBoard training curves](tb_curve.png)